<a href="https://colab.research.google.com/github/soumyadiptapete/Coursera_TF_specialization/blob/master/NLP_Week4_Exercise_Shakespeare_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

Using TensorFlow backend.


In [2]:
tokenizer = Tokenizer()
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

--2019-09-10 14:47:23--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2019-09-10 14:47:28 (118 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [4]:
embedding_dim=64
model = Sequential()
model.add(Embedding(total_words,embedding_dim,input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(200,activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words,activation='softmax'))
# Pick an optimizer
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 64)            205504    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 10, 100)           46000     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 100)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              645411    
Total params: 1,097,915
Trainable params: 1,097,915
Non-trainable params: 0
____________________________________________

In [5]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

W0910 14:48:23.322309 139820555270016 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/math_grad.py:1250: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
15462/15462 [==============================] - 52s 3ms/sample - loss: 6.9381 - acc: 0.0208
Epoch 2/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 6.5264 - acc: 0.0250
Epoch 3/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 6.3902 - acc: 0.0344
Epoch 4/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 6.2238 - acc: 0.0391
Epoch 5/100
15462/15462 [==============================] - 50s 3ms/sample - loss: 6.0272 - acc: 0.0504
Epoch 6/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 5.8239 - acc: 0.0623
Epoch 7/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 5.6270 - acc: 0.0744
Epoch 8/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 5.4309 - acc: 0.0874
Epoch 9/100
15462/15462 [==============================] - 49s 3ms/sample - loss: 5.2156 - acc: 0.1016
Epoch 10/100
15462/15462 [==============================] - 49s 3ms/sampl

In [1]:
import matplotlib.pyplot as plt
acc = history.history['acc']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()

NameError: ignored

In [0]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)